# Objective
The objective of this assignment is to develop a web scraper to extract specific information from an app/website. This assignment will test your ability to understand the app/web scraping concepts, make use of scraping libraries, and counter challenges such as blocking, authentication, etc., in the data extraction process.

## Problem Statement
Develop the scraper to scrape Grab Food Delivery.

**Website/App:** Grab food - Singapore

**Location:**
- PT Singapore - Choa Chu Kang North 6, Singapore, 689577
- Chong Boon Dental Surgery - Block 456 Ang Mo Kio Avenue 10, #01-1574, Singapore, 560456

**Tasks that scraper should perform:**
1. Extract the restaurant list of the selected location along with restaurant details (Please refer to the schema given below).
2. Create a unique restaurant list.
3. Extract the delivery fee and estimated delivery time for any one of the selected locations.



In [97]:
"""
Problem Statement: Develop the scraper to scrape Grab Food Delivery. 

"""

#Import the libraries 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
import requests
import re
import gzip
import json
import os

from selenium.webdriver.common.keys import Keys


## Defining clss for Grab food web scraper using oops concepts
class GrabFoodScraper:
    def __init__(self, url,location_input_text):
        """
        Initialize the GrabFoodScraper class with the given URL and location input text.

        """
        self.url = url
        self.location_input_text = location_input_text
        
        self.scraped_data = None

        self.chrome_options = webdriver.ChromeOptions()
        self._configure_chrome_options()

        self.driver = webdriver.Chrome(options=self.chrome_options)

    #Configure Chrome options, including setting a user agent, incognito mode, and disabling Blink features.
    def _configure_chrome_options(self):
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134"
        self.chrome_options.add_argument(f'user-agent={user_agent}')
        self.chrome_options.add_argument("--incognito")
        self.chrome_options.add_argument("--disable-blink-features=AutomationControlled")

    def _accept_cookies(self):
        """
        Accept cookies by clicking on the 'Accept' button if present.
        """
        try:
            WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Accept")]'))).click()
        except TimeoutException:
            pass

    def _scroll_to_bottom(self):
        """
        Scroll down to the bottom of the page to load more content dynamically.
        """
        last_height = self.driver.execute_script("return document.body.scrollHeight")

        while True:
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = self.driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    def _extract_restaurant_id(self, url):
        restaurant_id_pattern = re.compile(r"/([^/]+)$")
        restaurant_id_match = restaurant_id_pattern.search(url)

        if restaurant_id_match:
            return restaurant_id_match.group(1)
        else:
            return None

    def _get_latlng_from_api(self, restaurant_id):
        """
        This function will used to extract the required key from the json data.
        using https://portal.grab.com/foodweb/v2/merchants/4-C363VJ5UEXXFME? api URL
        BY COMBINING ID WITH BASE URL
        ALL THE MENTIONED FEATURES IS EXTRACTED HERE IN THIS FUNCTION 
        Restaurant Name
        Restaurant Cuisine
        Restaurant Rating
        Estimate time of Delivery
        Restaurant Distance from Delivery Location
        Promotional Offers Listed for the Restaurant.
        Restaurant Notice If Visible.
        Image Link of the Restaurant
        Is promo available (True/False)
        Restaurant ID (Look like this: 4-C2NKCETKWE5WLT)
        Restaurant latitude and longitude
        Estimate Delivery Fee

        """      
        api_url = f"https://portal.grab.com/foodweb/v2/merchants/{restaurant_id}"
        headers = {
            "authority": "portal.grab.com",
            "method": "GET",
            "scheme": "https",
            "accept": "application/json, text/plain, */*",
            "accept-encoding": "gzip, deflate, br",
            "accept-language": "en",
            "dnt": "1",
            "origin": "https://food.grab.com",
            "referer": "https://food.grab.com/",
            "sec-ch-ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": "Windows",
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-site",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
            "x-country-code": "SG",
            "x-gfc-country": "SG",
               }

        response = requests.get(api_url, headers=headers)
        print(response.status_code)

        if response.status_code == 200:
            data = response.json()
            latitude = data.get("merchant", {}).get("latlng", {}).get("latitude")
            longitude = data.get("merchant", {}).get("latlng", {}).get("longitude")
            name = data.get("merchant", {}).get("name")
            opening_hours = data.get("merchant", {}).get("openingHours", {})
            cuisines = data.get("merchant",{}).get("cuisine")

            IDs = data.get("merchant",{}).get("ID")


            rating = data.get("merchant",{}).get("rating")

            distances = data.get("merchant",{}).get("distanceInKm")

            img_url = data.get("merchant",{}).get("photoHref")

            menu = data.get("merchant", {}).get("menu", {})


            if latitude and longitude and name:
                print(f"Latitude: {latitude}, Longitude: {longitude}")
                print(f"Merchant Name: {name}")
                print(f"cusine : {cuisines}")
                print(f"Id : {IDs}")

                print(f"ratings : {rating}")
                print(f"dist : {distances}")

                print(f"img_url{img_url}")
                if opening_hours:
                    print("Opening Hours:")
                    for day, hours in opening_hours.items():
                        print(f"{day.capitalize()}: {hours}")
                    promo_availability = {}
                    try:
                        if menu:
                            print("Promo Availability:")
                            for category in menu.get('categories', []):
                                category_name = category.get('name')
                                category_items = category.get('items', [])

                                # Check availability for each item within the category
                                for item in category_items:
                                    item_name = item.get('name')
                                    item_available = item.get('available', False)
                                    print(f"{item_name}: {item_available}")
                                    promo_availability[item_name] = item_available
                        else:
                            print("Menu not found.")
                    except Exception as e:
                        print(f"An error occurred while processing menu data: {e}")
                        # Handle the exception as needed (log, raise, etc.)



                    return latitude, longitude, name, opening_hours,cuisines,promo_availability,IDs,rating,distances ,img_url
                else:
                    print("Opening Hours not found.")
                    return latitude, longitude, name, None, cuisines,promo_availability,IDs,ratings,distances , img_url
            else:
                print("Latitude and Longitude not found.")
                return None, None, None, None,None,None,None , None, None,None


        else:
            print("Error:", response.status_code)
            return None, None, None, None,None,None,None , None, None,None

       
    def scrape_grab_food(self):
        self.driver.get(self.url)
        self._accept_cookies()

        try:
            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ant-layout")))
        except TimeoutException:
            print("Timeout waiting for layout element.")
            return

        location_input = self.driver.find_element(By.ID, 'location-input')
        location_input.click()
        time.sleep(2)
        location_input.clear()
        loc = self.location_input_text
        location_input.send_keys(loc)

        print("Suggestion start")
        suggestion_items = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.ant-select-dropdown-menu-item')))
        print('Suggestion end ')

        if suggestion_items:
            self.driver.execute_script("arguments[0].click();", suggestion_items[0])

        submit_button = self.driver.find_element(By.CSS_SELECTOR, '.ant-btn.submitBtn___2roqB.ant-btn-primary') 
        submit_button.click()

        WebDriverWait(self.driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ant-layout")))

        last_height = self.driver.execute_script("return document.body.scrollHeight")

        while True:
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = self.driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        layout_div = self.driver.find_element(By.CSS_SELECTOR, '.ant-layout')
        restaurant_names = layout_div.find_elements(By.CSS_SELECTOR, '.name___2epcT')
        names = [name.text for name in restaurant_names]
        names  = names[10:]
        
        ##creating pandas data frame to storing the Restaurants data and URLs
        db = pd.DataFrame(names, columns=['Restaurant'])
        url_elements = self.driver.find_elements(By.XPATH, "//a[contains(@href, '/sg/en/restaurant')]")
        url_elements_list = [url_element.get_attribute("href") for url_element in url_elements][10:]
        db_temp = pd.DataFrame(url_elements_list, columns=['url'])
        db = pd.concat([db, db_temp], axis=1, join='inner')

        db['Latitude'] = ''
        db['Longitude'] = ''
        db['name'] = ''
        db['Opening Hours'] = ''
        db['cuisine'] = ''
        db['promo'] = ''
        db['ID'] = ''
        db['rating'] = ''
        db['distanceKm'] = ''
        db['photoHref'] = ''

        i = 0
        for index, row in db.iterrows():
            restaurant_url = row['url']
            restaurant_name = row['Restaurant']

            restaurant_id = self._extract_restaurant_id(restaurant_url)

            if restaurant_id:
                restaurant_latitude, restaurant_longitude, name, opening_hours, cuisines, promo_availability, IDs, rating, distances, img_url = self._get_latlng_from_api(restaurant_id)
                db.at[i, 'Latitude'] = restaurant_latitude
                db.at[i, 'Longitude'] = restaurant_longitude
                db.at[i, 'name'] = name
                db.at[i, 'Opening Hours'] = opening_hours
                db.at[i, 'cuisine'] = cuisines
                db.at[i, 'promo'] = promo_availability
                db.at[i, 'ID'] = IDs
                db.at[i, 'rating'] = rating
                db.at[i, 'distanceKm'] = distances
                db.at[i, 'photoHref'] = img_url

            time.sleep(5)
            i += 1

        display(db)
        self.scraped_data = db
        data = db.to_dict(orient='records')
        
        current_directory = os.getcwd()
        output_file_path = os.path.join(current_directory, 'output.ndjson.gz')

        with gzip.open(output_file_path, 'wb') as f:
            for record in data:
                json_string = json.dumps(record, ensure_ascii=False)
                f.write(json_string.encode('utf-8'))
                
        db.to_excel('1.xlsx', index=False)

        self.driver.quit()
        
    def display_db(self):
        # Check if data is available
        if self.scraped_data is not None:
            display(self.scraped_data)
        else:
            print("No data available. Please run scrape_grab_food first.")

#Using GrabFoodScraper Class and passing the website base url and the location for which we want to extract information 
grab_food_scraper = GrabFoodScraper("https://food.grab.com/sg/en","PT Singapore - Choa Chu Kang North 6, Singapore, 689577")
grab_food_scraper.scrape_grab_food()


#### FOR THE SECOND LOCATION 
# grab_food_scraper_LOCATION_2 = GrabFoodScraper("https://food.grab.com/sg/en","Chong Boon Dental Surgery - Block 456 Ang Mo Kio Avenue 10, #01-1574, Singapore, 560456")
# grab_food_scraper_LOCATION_2.scrape_grab_food()

Suggestion start
Suggestion end 
200
Latitude: 1.392342988, Longitude: 103.7430223
Merchant Name: KOI Thé - Limbang Shopping Centre
cusine : Bubble Tea,Drinks & Beverages
Id : 4-C363VJ5UEXXFME
ratings : 4.7
dist : 20.68
img_urlhttps://d1sag4ddilekf6.cloudfront.net/compressed_webp/merchants/4-C363VJ5UEXXFME/hero/5a28f6dc-39b2-4d32-9127-da7aa9ea0f56__store_cover__2023__12__28__05__11__55.webp
Opening Hours:
Open: True
Displayedhours: 11:00-21:45
Sun: 11:00am-9:45pm
Mon: 11:00am-9:45pm
Tue: 11:00am-9:45pm
Wed: 11:00am-9:45pm
Thu: 11:00am-9:45pm
Fri: 11:00am-9:45pm
Sat: 11:00am-9:45pm
Promo Availability:
M-Jasmine Green Tea: True
L-Jasmine Green Tea: True
M-Honey Green Tea: True
L-Honey Green Tea: True
M-Peach Green Tea: True
L-Peach Green Tea: True
M-Yakult Gold Green Tea: True
L-Yakult Gold Green Tea: True
M-Lemon Green Tea: True
L-Lemon Green Tea: True
M-Fresh Passion Green Tea: True
L-Fresh Passion Green Tea: True
M-Ai Yu Iced Tea: True
L-Ai Yu Iced Tea: True
M-Assam Black Tea: True
L-

200
Latitude: 1.3974199256833089, Longitude: 103.7467085782157
Merchant Name: Nine Fresh - Yewtee Point
cusine : Bubble Tea,Dessert,Drinks & Beverages
Id : 4-C4NGLLMWDE2YAE
ratings : 4.5
dist : 20.987
img_urlhttps://d1sag4ddilekf6.cloudfront.net/compressed_webp/merchants/4-C4NGLLMWDE2YAE/hero/c57b0a59-627e-4956-9de8-f95970baed0b__store_cover__2023__11__03__01__59__56.webp
Opening Hours:
Open: True
Displayedhours: 11:30-21:30
Sun: 11:30am-9:30pm
Mon: 11:30am-9:30pm
Tue: 11:30am-9:30pm
Wed: 11:30am-9:30pm
Thu: 11:30am-9:30pm
Fri: 11:30am-9:30pm
Sat: 11:30am-9:30pm
Promo Availability:
Chrysanthemum Ai-Yu A 菊花爱玉蜂蜜泡泡葡萄晶球: True
Chrysanthemum Ai-Yu B 菊花爱玉蜂蜜泡泡小芋头: True
For 1: True
For 2: True
Create Your Own Specials: True
Red Tea 红茶: True
Jasmine Green Tea  茉香绿茶: True
Four Seasons Oolong Tea  四季春乌龙茶: True
Earl Grey Tea  格雷伯爵茶: True
Milk Tea  奶茶: True
Earl Grey Milk Tea  格雷伯爵奶茶: True
1. Grass Jelly Special 黑糖芦荟仙草冻: True
2. Oat Treasures Grass Jelly 八宝燕麦仙草冻: True
3. Nine Fresh Signature 九鲜招牌: T

200
Latitude: 1.3970125569379337, Longitude: 103.746636609208
Merchant Name: Sarawak Kolo Mee - Yewtee Point
cusine : Chinese,Noodles,Local & Malaysian
Id : 4-C6DAN6DAAT53AN
ratings : 4
dist : 20.815
img_urlhttps://d1sag4ddilekf6.cloudfront.net/compressed_webp/merchants/4-C6DAN6DAAT53AN/hero/4e4c1a1994474b67aede952dd8e7d072_1706081706678273683.webp
Opening Hours:
Open: True
Displayedhours: 09:00-20:30
Sun: 9:00am-8:30pm
Mon: 9:00am-8:30pm
Tue: 9:00am-8:30pm
Wed: 9:00am-8:30pm
Thu: 9:00am-8:30pm
Fri: 9:00am-8:30pm
Sat: 9:00am-8:30pm
Promo Availability:
luncheon meat 午餐肉 6pcs: True
fried chicken wing-鸡中翅 6pcs: True
Sarawak Wanton Soup 砂拉越云吞汤 12pcs: True
Kuching Kolo Mee  古晋哥罗面: True
Sibu Mee Kampua干盘面: True
Kuching Laksa 古晋叻沙: True
Sarawak Wanton Noodle 砂拉越云吞面: True
Sarawak Pork Noodle 砂拉越肉片面: True
Pork Liver Noodle 猪肝面: True
Crispy Chicken Kolo Mee 脆皮鸡哥罗面: True
Sarawak Sheng Liao Noodle 砂拉越生料面: True
Kuching Three Color Noodle 古晋三色面: True
Luncheon Meat & Enoki Mushroom Kolo Mee 午餐肉金针菇: T

200
Latitude: 1.3970582, Longitude: 103.7467252
Merchant Name: Saizeriya - Yew Tee Point #01-17
cusine : Italian,Pasta,Pizza
Id : 4-CZEAR4NABCAUAA
ratings : 4.5
dist : 20.861
img_urlhttps://d1sag4ddilekf6.cloudfront.net/compressed_webp/merchants/4-CZEAR4NABCAUAA/hero/81ce1c22f60f4e4d9a07007a345fb344_1669173850370617746.webp
Opening Hours:
Open: True
Displayedhours: 11:00-21:00
Sun: 11:00am-9:00pm
Mon: 11:00am-9:00pm
Tue: 11:00am-9:00pm
Wed: 11:00am-9:00pm
Thu: 11:00am-9:00pm
Fri: 11:00am-9:00pm
Sat: 11:00am-9:00pm
Promo Availability:
Mushroom Soup: True
Corn Cream Soup: True
Tomato Cream Soup: True
Grill Combo: True
Chicken Wing 8Pcs: True
Chicken Wing - 5Pcs: True
.Crispy Chicken 12pcs: True
Crispy Chicken: True
Cheesy Curry Potato: True
Sauted Spinach: True
Sauteed Broccoli: True
Butter Corn: True
Cream Asparagus with Bacon: True
Grill Green Mussel: True
Baked Asari Clam: True
Beddar Cheddar Sausage: True
Original Focaccia: True
Garlic Focaccia: True
Cheesy Focaccia: True
Chicken Win

200
Latitude: 1.397419926, Longitude: 103.7467086
Merchant Name: Takagi Ramen - Yew Tee Point
cusine : Ramen,Noodles,Japanese
Id : 4-C36AVPC3NCJEWA
ratings : 4.1
dist : 21.154
img_urlhttps://d1sag4ddilekf6.cloudfront.net/compressed_webp/merchants/4-C36AVPC3NCJEWA/hero/7f0de0949c804ab3a906b85695c9adb2_1707275393558211292.webp
Opening Hours:
Open: True
Displayedhours: 00:00-23:59
Sun: 12:00am-11:59pm
Mon: 12:00am-11:59pm
Tue: 12:00am-11:59pm
Wed: 12:00am-11:59pm
Thu: 12:00am-11:59pm
Fri: 12:00am-11:59pm
Sat: 12:00am-11:59pm
Promo Availability:
Konnyaku Tomato Ramen 🍅🍜: True
Bekon Pumpkin Ramen 🎃🍜: True
Joyful Lychee Ice Blended 🍚: True
Lucky Pineapple Ice Blended 🍍: True
Konnyaku Tomato Ramen Combo 🍚🍍🍜: True
Bekon Pumpkin Ramen Combo 🍚🍍🍜: True
Uji Matcha Mochi: True
Yoshino Sakura Mochi: True
Family Bundle Buddy Meal for 4Pax: True
Family Bundle Buddy Meal for 6Pax: True
Miso Shoyu Ramen 20% OFF! (U.P. $10.90): True
🍗Chicken Karaage (6pcs) 20% OFF! (U.P. $7.90): True
Local Kopi☕[Hot/Cold

...

200
Latitude: 1.3850869, Longitude: 103.7434528
Merchant Name: Golden Chicken - Choa Chu Kang Ave 4
cusine : Local & Malaysian,Chinese,Noodles
Id : 4-CYWTPGDYJN3FEN
ratings : 3.8
dist : 20.902
img_urlhttps://d1sag4ddilekf6.cloudfront.net/compressed_webp/merchants/4-CYWTPGDYJN3FEN/hero/696a376705164edb891498e5b2984523_1567069878982851158.webp
Opening Hours:
Displayedhours: Closed
Sun: 6:00am-7:00am
Mon: Closed
Tue: Closed
Wed: Closed
Thu: Closed
Fri: Closed
Sat: Closed
Promo Availability:
Chicken Rice Set 鸡饭套餐: True
Chicken Rice 鸡饭: True
3 Combination Rice 3拼饭: True
Thai Style Beancurd 泰式豆腐: True
Chicken 鸡: True
Wanton Mee 云吞面: False
Shredded Chicken Hor Fun 鸡丝河粉: False
200
Latitude: 1.384633, Longitude: 103.744728
Merchant Name: Tori Story By I Love Taimei - Lot One, Shoppers' Mall
cusine : Bubble Tea,Chinese,Fried Chicken
Id : 4-C6D2WFTFCTCKRX
ratings : None
dist : 20.596
img_urlhttps://d1sag4ddilekf6.cloudfront.net/compressed_webp/merchants/4-C6D2WFTFCTCKRX/hero/7e5714a298fb4c8993e25

200
Latitude: 1.38318885229917, Longitude: 103.740320319592
Merchant Name: Al Rsak Restaurant - Choa Chu Kang Avenue 4
cusine : Indian,Fried Chicken,Local & Malaysian
Id : 4-C35VBFDCVXU3WA
ratings : None
dist : 21.222
img_urlhttps://d1sag4ddilekf6.cloudfront.net/compressed_webp/merchants/4-C35VBFDCVXU3WA/hero/1d7e8e1f321d417797cb3b07c18ab2de_1681115304779000329.webp
Opening Hours:
Displayedhours: Closed
Sun: 12:00am-11:59pm
Mon: 12:00am-11:59pm
Tue: 12:00am-11:59pm
Wed: 12:00am-11:59pm
Thu: 12:00am-11:59pm
Fri: 12:00am-11:59pm
Sat: 12:00am-11:59pm
Tempclosed: True
Promo Availability:
Plain Prata  (2 Pieces): True
Paper Prata: True
Onion Prata: True
Egg Prata: True
Plaster Prata: True
Egg Onion Prata: True
Double Egg Prata: True
Cheese Prata: True
Mushroon Prata: True
Egg Mushroom Prata: True
Butter Prata: True
Prata Bomb: True
Honey Prata: True
Garlic Prata: True
Hotdog Prata: True
Milk Prata: True
Banana Prata: True
Horlicks Prata: True
Sausage: True
Strawberry: True
Egg Sausage: True

200
Latitude: 1.3814445, Longitude: 103.752292
Merchant Name: Cantolicious - Teck Whye Ave
cusine : Chinese,Noodles,Zi Char
Id : SGDD09475
ratings : 3.7
dist : 18.464
img_urlhttps://d1sag4ddilekf6.cloudfront.net/compressed_webp/merchants/SGDD09475/hero/bc3ca7ed57ee40c0b6e7ca169eda6f58_1572330563619822961.webp
Opening Hours:
Displayedhours: Closed
Sun: 11:00am-9:00pm
Mon: 11:00am-9:00pm
Tue: 11:00am-9:00pm
Wed: 11:00am-9:00pm
Thu: 11:00am-9:00pm
Fri: 11:00am-9:00pm
Sat: 11:00am-9:00pm
Tempclosed: True
Promo Availability:
1. 扬州炒饭 Yangzhou Fried Rice: True
2. 黄梨炒饭 Pineapple Fried Rice: True
3. 咸鱼炒饭 Salted Fish Fried Rice: True
4. 叁笆炒饭 Sambal Fried Rice: True
5. 海鲜炒饭 Seafood Fried Rice: True
6. 烩饭 Cantonese ‘ Mui Fan’: True
7. 干炒河粉 Dry Fried Horfun: True
8. 什锦河粉 Horfun in Seafood Sauce: True
9. 新加坡炒福建面 Hokkien Mee: True
10. 馬來叁芭炒面 Mee Goreng: True
11. 鱼片米粉 Sliced Fish Beehoon (Regular): True
12. 三捞河粉 ‘San Lao’ (Sliced Fish) Horfun: True
13. 鱼头米粉 Sliced Fish Head Beehoon (Regular): False
14

200
Latitude: 1.381322186712552, Longitude: 103.75249035182077
Merchant Name: Traditional Taste 古早味 - Teck Whye Ave 
cusine : Chinese,Local & Malaysian,Breakfast & Brunch
Id : 4-C2TDN7JXJLBBFE
ratings : 4.1
dist : 18.473
img_urlhttps://d1sag4ddilekf6.cloudfront.net/compressed_webp/merchants/4-C2TDN7JXJLBBFE/hero/145c6aab8c1a4987943fc479fea8f366_1623317059774524666.webp
Opening Hours:
Displayedhours: Closed
Sun: 6:00am-12:00pm 5:00pm-9:00pm
Mon: 6:00am-12:00pm 5:00pm-9:00pm
Tue: 6:00am-12:00pm
Wed: 6:00am-12:00pm 5:00pm-9:00pm
Thu: 6:00am-12:00pm 5:00pm-9:00pm
Fri: 6:00am-12:00pm 5:00pm-9:00pm
Sat: 6:00am-12:00pm 5:00pm-9:00pm
Promo Availability:
Nasi Lemak Set 1 椰浆饭(套餐一): False
Nasi Lemak Set 2 椰浆饭 (套餐二): False
Nasi Lemak Set 3 椰浆饭(套餐三): False
Bee Hoon Set 米粉套餐: False
Fried Hokkien Mee 炒福建面: False
Fried White Bee Hoon 炒白米粉: False
Fried Maggie Noodle 炒伊面: False
Fried Bee Hoon 炒米粉: False
Fried Noodle 炒面: False
Chicken Wing 鸡翅膀: False
Chicken Katsudon 鸡扒: False
Fish Bian Xiang 扁香: False
T

,Restaurant,url,Latitude,Longitude,name,Opening Hours,cuisine,promo,ID,rating,distanceKm,photoHref
0,KOI Thé - Limbang Shopping Centre,https://food.grab.com/sg/en/restaurant/koi-th%...,1.392343,103.743022,KOI Thé - Limbang Shopping Centre,"{'open': True, 'displayedHours': '11:00-21:45'...","Bubble Tea,Drinks & Beverages","{'M-Jasmine Green Tea': True, 'L-Jasmine Green...",4-C363VJ5UEXXFME,4.7,20.68,https://d1sag4ddilekf6.cloudfront.net/compress...
1,D'Laksa - Yewtee Point,https://food.grab.com/sg/en/restaurant/d-laksa...,1.397013,103.746637,D'Laksa - Yewtee Point,"{'open': True, 'displayedHours': '10:30-21:30'...","Local & Malaysian,Snack,Drinks & Beverages","{'D'Light Deal for 2 with Drinks': True, 'Asam...",4-C6CXT7CVA3EXAT,4.2,20.815,https://d1sag4ddilekf6.cloudfront.net/compress...
2,Wingstop - Limbang Shopping Centre,https://food.grab.com/sg/en/restaurant/wingsto...,1.391957,103.743614,Wingstop - Limbang Shopping Centre,"{'open': True, 'displayedHours': '11:00-21:45'...","Fast Food,Fried Chicken,Western,Drinks & Bever...","{'Pandan Churros with Gula Melaka Dip': True, ...",4-C4LBG4JVR4B2AA,4.3,20.685,https://d1sag4ddilekf6.cloudfront.net/compress...
3,KFC - Yew Tee Point,https://food.grab.com/sg/en/restaurant/kfc-yew...,1.396962,103.746696,KFC - Yew Tee Point,"{'open': True, 'displayedHours': '06:15-11:00 ...","Fast Food,Fried Chicken,Halal",{'<Coke Promo> 5pcs Chicken Buddy Meal + 2DIP'...,4-CY3TDGLTA7CAA2,4.1,20.674,https://d1sag4ddilekf6.cloudfront.net/compress...
4,Nine Fresh - Yewtee Point,https://food.grab.com/sg/en/restaurant/nine-fr...,1.39742,103.746709,Nine Fresh - Yewtee Point,"{'open': True, 'displayedHours': '11:30-21:30'...","Bubble Tea,Dessert,Drinks & Beverages","{'Chrysanthemum Ai-Yu A 菊花爱玉蜂蜜泡泡葡萄晶球': True, '...",4-C4NGLLMWDE2YAE,4.5,20.987,https://d1sag4ddilekf6.cloudfront.net/compress...
...,...,...,...,...,...,...,...,...,...,...,...,...
253,Wee O'Clock - Choa Chu Kang Ave 4,https://food.grab.com/sg/en/restaurant/wee-o-c...,1.381713,103.74147,Wee O'Clock - Choa Chu Kang Ave 4,"{'displayedHours': 'Closed', 'sun': 'Closed', ...","Bubble Tea,Snack,Drinks & Beverages",{'[COMBO] Japanese Original Mille Crepe Cake':...,4-C3CCCPXFGY3VHE,4.6,20.698,https://d1sag4ddilekf6.cloudfront.net/compress...
254,Traditional Taste 古早味 - Teck Whye Ave,https://food.grab.com/sg/en/restaurant/traditi...,1.381322,103.75249,Traditional Taste 古早味 - Teck Whye Ave,"{'displayedHours': 'Closed', 'sun': '6:00am-12...","Chinese,Local & Malaysian,Breakfast & Brunch","{'Nasi Lemak Set 1 椰浆饭(套餐一)': False, 'Nasi Lem...",4-C2TDN7JXJLBBFE,4.1,18.473,https://d1sag4ddilekf6.cloudfront.net/compress...
255,Mandai Coffee Shop - Mandai Connection,https://food.grab.com/sg/en/restaurant/mandai-...,1.408948,103.757212,Mandai Coffee Shop - Mandai Connection,"{'displayedHours': 'Closed', 'sun': '10:00am-9...","Local & Malaysian,Mala,Chinese","{'Mild Spicy 少辣，': True, 'Medium Spicy 中辣，': T...",4-C32ZV2D2PEXJCE,3,22.397,https://d1sag4ddilekf6.cloudfront.net/compress...
256,Stickies @ Sungei Kadut - Sungei Kadut Industr...,https://food.grab.com/sg/en/restaurant/stickie...,1.41226,103.74843,Stickies @ Sungei Kadut - Sungei Kadut Industr...,"{'displayedHours': 'Closed', 'sun': '3:00pm-10...",Alcohol,"{'Hana Peach': True, 'Hana Watermelon': True, ...",4-C3XFTYWUELJUR6,None,22.965,https://d1sag4ddilekf6.cloudfront.net/compress...


In [98]:
grab_food_scraper.display_db()


,Restaurant,url,Latitude,Longitude,name,Opening Hours,cuisine,promo,ID,rating,distanceKm,photoHref
0,KOI Thé - Limbang Shopping Centre,https://food.grab.com/sg/en/restaurant/koi-th%...,1.392343,103.743022,KOI Thé - Limbang Shopping Centre,"{'open': True, 'displayedHours': '11:00-21:45'...","Bubble Tea,Drinks & Beverages","{'M-Jasmine Green Tea': True, 'L-Jasmine Green...",4-C363VJ5UEXXFME,4.7,20.68,https://d1sag4ddilekf6.cloudfront.net/compress...
1,D'Laksa - Yewtee Point,https://food.grab.com/sg/en/restaurant/d-laksa...,1.397013,103.746637,D'Laksa - Yewtee Point,"{'open': True, 'displayedHours': '10:30-21:30'...","Local & Malaysian,Snack,Drinks & Beverages","{'D'Light Deal for 2 with Drinks': True, 'Asam...",4-C6CXT7CVA3EXAT,4.2,20.815,https://d1sag4ddilekf6.cloudfront.net/compress...
2,Wingstop - Limbang Shopping Centre,https://food.grab.com/sg/en/restaurant/wingsto...,1.391957,103.743614,Wingstop - Limbang Shopping Centre,"{'open': True, 'displayedHours': '11:00-21:45'...","Fast Food,Fried Chicken,Western,Drinks & Bever...","{'Pandan Churros with Gula Melaka Dip': True, ...",4-C4LBG4JVR4B2AA,4.3,20.685,https://d1sag4ddilekf6.cloudfront.net/compress...
3,KFC - Yew Tee Point,https://food.grab.com/sg/en/restaurant/kfc-yew...,1.396962,103.746696,KFC - Yew Tee Point,"{'open': True, 'displayedHours': '06:15-11:00 ...","Fast Food,Fried Chicken,Halal",{'<Coke Promo> 5pcs Chicken Buddy Meal + 2DIP'...,4-CY3TDGLTA7CAA2,4.1,20.674,https://d1sag4ddilekf6.cloudfront.net/compress...
4,Nine Fresh - Yewtee Point,https://food.grab.com/sg/en/restaurant/nine-fr...,1.39742,103.746709,Nine Fresh - Yewtee Point,"{'open': True, 'displayedHours': '11:30-21:30'...","Bubble Tea,Dessert,Drinks & Beverages","{'Chrysanthemum Ai-Yu A 菊花爱玉蜂蜜泡泡葡萄晶球': True, '...",4-C4NGLLMWDE2YAE,4.5,20.987,https://d1sag4ddilekf6.cloudfront.net/compress...
...,...,...,...,...,...,...,...,...,...,...,...,...
253,Wee O'Clock - Choa Chu Kang Ave 4,https://food.grab.com/sg/en/restaurant/wee-o-c...,1.381713,103.74147,Wee O'Clock - Choa Chu Kang Ave 4,"{'displayedHours': 'Closed', 'sun': 'Closed', ...","Bubble Tea,Snack,Drinks & Beverages",{'[COMBO] Japanese Original Mille Crepe Cake':...,4-C3CCCPXFGY3VHE,4.6,20.698,https://d1sag4ddilekf6.cloudfront.net/compress...
254,Traditional Taste 古早味 - Teck Whye Ave,https://food.grab.com/sg/en/restaurant/traditi...,1.381322,103.75249,Traditional Taste 古早味 - Teck Whye Ave,"{'displayedHours': 'Closed', 'sun': '6:00am-12...","Chinese,Local & Malaysian,Breakfast & Brunch","{'Nasi Lemak Set 1 椰浆饭(套餐一)': False, 'Nasi Lem...",4-C2TDN7JXJLBBFE,4.1,18.473,https://d1sag4ddilekf6.cloudfront.net/compress...
255,Mandai Coffee Shop - Mandai Connection,https://food.grab.com/sg/en/restaurant/mandai-...,1.408948,103.757212,Mandai Coffee Shop - Mandai Connection,"{'displayedHours': 'Closed', 'sun': '10:00am-9...","Local & Malaysian,Mala,Chinese","{'Mild Spicy 少辣，': True, 'Medium Spicy 中辣，': T...",4-C32ZV2D2PEXJCE,3,22.397,https://d1sag4ddilekf6.cloudfront.net/compress...
256,Stickies @ Sungei Kadut - Sungei Kadut Industr...,https://food.grab.com/sg/en/restaurant/stickie...,1.41226,103.74843,Stickies @ Sungei Kadut - Sungei Kadut Industr...,"{'displayedHours': 'Closed', 'sun': '3:00pm-10...",Alcohol,"{'Hana Peach': True, 'Hana Watermelon': True, ...",4-C3XFTYWUELJUR6,None,22.965,https://d1sag4ddilekf6.cloudfront.net/compress...
